In [5]:
%pip install rouge


Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper


from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

True

In [12]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model='gpt-4o',
    temperature=0 
)


retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 200,
        "lambda_mult": 0.5,
        # "filters": {"리본개수": {"$gte": 0}}
    }
)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        당신은 한국의 식당을 소개하는 인공지능 비서입니다. 
        반드시 질문에 대해서 [context]에 주어진 내용을 바탕으로 답변을 해주세요. 
        질문에 '리본개수', '평점', '몇 개'라는 키워드가 포함된 경우, [context]에서 "리본개수" 항목을 확인해 답변하세요.
        리본개수는 평점과 같은 의미를 가집니다.
        [context]
        {context}
    """)),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

def content_from_doc(docs:list[Document]):
    return "\n\n".join([d.page_content for d in docs])


chain =  {'context': retriever  | RunnableLambda(content_from_doc), 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()

In [15]:
QUERY = "예약 가능한 한식당 추천해줘."
response = chain.invoke(QUERY)
print(response)

다음은 예약 가능한 한식당입니다:

1. **상주식당**
   - 음식종류: 한식(민물어패류), 추어탕
   - 전화번호: 053-425-5924
   - 주소: 대구광역시 중구 국채보상로 598-1
   - 메뉴: 추어탕+밥(1만2천원)

2. **이슬회수산**
   - 음식종류: 한식(어패류), 생선회, 조개찜
   - 전화번호: 041-633-4857
   - 주소: 충청남도 홍성군 서부면 남당항로213번길 1
   - 메뉴: 해물탕, 모둠회, 농어, 조개찜세트

3. **대한식당**
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 주소: 전라남도 광양시 광양읍 매일시장길 12-15
   - 메뉴: 국내산광양불고기, 호주산광양불고기, 소특양구이

4. **연춘식당**
   - 음식종류: 한식(민물어패류), 장어, 한식(가금류), 닭구이
   - 전화번호: 041-545-2866
   - 주소: 충청남도 아산시 신정호길 67
   - 메뉴: 장어구이, 닭구이, 닭고기야채볶음, 된장찌개

5. **옥야식당**
   - 음식종류: 한식(탕/국/찌개/전골), 선지해장국
   - 전화번호: 054-853-6953
   - 주소: 경상북도 안동시 중앙시장길 7
   - 메뉴: 선지국밥, 포장

이 외에도 다양한 예약 가능한 한식당이 있으니, 특정 지역이나 메뉴를 원하시면 추가로 알려주세요.


In [ ]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model='gpt-4o',
    temperature=0 
)


retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 200,
        "lambda_mult": 0.5,
        # "filters": {"리본개수": {"$gte": 0}}
    }
)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        당신은 한국의 식당을 소개하는 인공지능 비서입니다. 
        반드시 질문에 대해서 [context]에 주어진 내용을 바탕으로 답변을 해주세요. 
        질문에 '리본개수', '평점', '몇 개'라는 키워드가 포함된 경우, [context]에서 "리본개수" 항목을 확인해 답변하세요.
        리본개수는 평점과 같은 의미를 가집니다.
        [context]
        {context}
    """)),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

def content_from_doc(docs:list[Document]):
    return "\n\n".join([d.page_content for d in docs])


chain =  {'context': retriever  | RunnableLambda(content_from_doc), 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()

# QUERY 실행 및 응답 확인
QUERY = "예약 가능한 한식당 추천해줘."
response = chain.invoke(QUERY)
print("Generated Response:", response)

# 테스트 데이터 초기화
initial_test_data = [
    {"question": "예약 가능한 한식당 추천해줘.", "expected_answer": ""},
    {"question": "리본개수가 2개인 식당을 추천해주세요.", "expected_answer": ""},
    {"question": "프랑스식을 추천해주세요.", "expected_answer": ""}
]

# expected_answer 생성
def generate_expected_answers(chain, test_data):
    for test_case in test_data:
        question = test_case["question"]
        response = chain.invoke(question)
        test_case["expected_answer"] = response.strip()
    return test_data

test_data = generate_expected_answers(chain, initial_test_data)

# 성능 평가 함수
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import pandas as pd

def evaluate_chain(chain, test_data):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = []

    for test_case in test_data:
        question = test_case["question"]
        expected_answer = test_case["expected_answer"]

        response = chain.invoke(question)
        generated_answer = response.strip()

        correct = expected_answer in generated_answer
        bleu_score = sentence_bleu([expected_answer.split()], generated_answer.split())
        rouge_scores = scorer.score(generated_answer, expected_answer)

        results.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "correct": correct,
            "bleu_score": bleu_score,
            "rouge": rouge_scores
        })

    return results

# 평가 실행
results = evaluate_chain(chain, test_data)

# 결과 출력
for result in results:
    print(f"Question: {result['question']}")
    print(f"Expected: {result['expected_answer']}")
    print(f"Generated: {result['generated_answer']}")
    print(f"Correct: {result['correct']}")
    print(f"BLEU Score: {result['bleu_score']:.2f}")
    print(f"ROUGE: {result['rouge']}")
    print("-" * 30)

# 결과 저장
df_results = pd.DataFrame(results)
df_results.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")


In [21]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model='gpt-4o',
    temperature=0 
)


retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 200,
        "lambda_mult": 0.5,
        # "filters": {"리본개수": {"$gte": 0}}
    }
)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        당신은 한국의 식당을 소개하는 인공지능 비서입니다. 
        반드시 질문에 대해서 [context]에 주어진 내용을 바탕으로 답변을 해주세요. 
        질문에 '리본개수', '평점', '몇 개'라는 키워드가 포함된 경우, [context]에서 "리본개수" 항목을 확인해 답변하세요.
        리본개수는 평점과 같은 의미를 가집니다.
        [context]
        {context}
    """)),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

def content_from_doc(docs:list[Document]):
    return "\n\n".join([d.page_content for d in docs])


chain =  {'context': retriever  | RunnableLambda(content_from_doc), 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()

# QUERY 실행 및 응답 확인
QUERY = "예약 가능한 한식당 추천해줘."
response = chain.invoke(QUERY)
print("Generated Response:", response)

# 테스트 데이터 초기화
initial_test_data = [
    {"question": "예약 가능한 한식당 추천해줘.", "expected_answer": ""},
    {"question": "리본개수가 2개인 식당을 추천해주세요.", "expected_answer": ""},
    {"question": "프랑스식을 추천해주세요.", "expected_answer": ""}
]

# expected_answer 생성
def generate_expected_answers(chain, test_data):
    for test_case in test_data:
        question = test_case["question"]
        response = chain.invoke(question)
        test_case["expected_answer"] = response.strip()
    return test_data

test_data = generate_expected_answers(chain, initial_test_data)

# 성능 평가 함수
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import pandas as pd

def evaluate_chain(chain, test_data):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = []

    for test_case in test_data:
        question = test_case["question"]
        expected_answer = test_case["expected_answer"]

        response = chain.invoke(question)
        generated_answer = response.strip()

        correct = expected_answer in generated_answer
        bleu_score = sentence_bleu([expected_answer.split()], generated_answer.split())
        rouge_scores = scorer.score(generated_answer, expected_answer)

        results.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "correct": correct,
            "bleu_score": bleu_score,
            "rouge": rouge_scores
        })

    return results

# 평가 실행
results = evaluate_chain(chain, test_data)

# 결과 출력
for result in results:
    print(f"Question: {result['question']}")
    print(f"Expected: {result['expected_answer']}")
    print(f"Generated: {result['generated_answer']}")
    print(f"Correct: {result['correct']}")
    print(f"BLEU Score: {result['bleu_score']:.2f}")
    print(f"ROUGE: {result['rouge']}")
    print("-" * 30)

# 결과 저장
df_results = pd.DataFrame(results)
df_results.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")

Generated Response: 다음은 예약 가능한 한식당입니다:

1. **상주식당**
   - 음식종류: 한식(민물어패류), 추어탕
   - 전화번호: 053-425-5924
   - 주소: 대구광역시 중구 국채보상로 598-1
   - 메뉴: 추어탕+밥(1만2천원)

2. **이슬회수산**
   - 음식종류: 한식(어패류), 생선회, 조개찜
   - 전화번호: 041-633-4857
   - 주소: 충청남도 홍성군 서부면 남당항로213번길 1
   - 메뉴: 해물탕, 모둠회, 농어, 조개찜세트

3. **대한식당**
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 주소: 전라남도 광양시 광양읍 매일시장길 12-15
   - 메뉴: 국내산광양불고기, 호주산광양불고기, 소특양구이

4. **연춘식당**
   - 음식종류: 한식(민물어패류), 장어, 한식(가금류), 닭구이
   - 전화번호: 041-545-2866
   - 주소: 충청남도 아산시 신정호길 67
   - 메뉴: 장어구이, 닭구이, 닭고기야채볶음, 된장찌개

5. **애성회관**
   - 음식종류: 한식(탕/국/찌개/전골), 곰탕, 한식(육류), 수육
   - 전화번호: 02-352-0303
   - 주소: 서울특별시 중구 남대문로5길 23
   - 메뉴: 곰탕, 한우와낙지, 낙지볶음, 한우수육, 한우불고기

이 외에도 다양한 예약 가능한 한식당이 있으니, 특정 지역이나 메뉴를 원하시면 추가로 알려주세요.


c:\Users\qkrck\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Question: 예약 가능한 한식당 추천해줘.
Expected: 다음은 예약 가능한 한식당입니다:

1. **상주식당**
   - 음식종류: 한식(민물어패류), 추어탕
   - 전화번호: 053-425-5924
   - 주소: 대구광역시 중구 국채보상로 598-1

2. **이슬회수산**
   - 음식종류: 한식(어패류), 생선회, 조개찜
   - 전화번호: 041-633-4857
   - 주소: 충청남도 홍성군 서부면 남당항로213번길 1

3. **대한식당**
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 주소: 전라남도 광양시 광양읍 매일시장길 12-15

4. **연춘식당**
   - 음식종류: 한식(민물어패류), 장어, 한식(가금류), 닭구이
   - 전화번호: 041-545-2866
   - 주소: 충청남도 아산시 신정호길 67

5. **옥야식당**
   - 음식종류: 한식(탕/국/찌개/전골), 선지해장국
   - 전화번호: 054-853-6953
   - 주소: 경상북도 안동시 중앙시장길 7

이 외에도 다양한 한식당이 있으니, 원하는 지역이나 메뉴에 따라 선택하시면 좋겠습니다.
Generated: 다음은 예약 가능한 한식당입니다:

1. **상주식당**
   - 음식종류: 한식(민물어패류), 추어탕
   - 전화번호: 053-425-5924
   - 주소: 대구광역시 중구 국채보상로 598-1
   - 메뉴: 추어탕+밥(1만2천원)

2. **이슬회수산**
   - 음식종류: 한식(어패류), 생선회, 조개찜
   - 전화번호: 041-633-4857
   - 주소: 충청남도 홍성군 서부면 남당항로213번길 1
   - 메뉴: 해물탕, 모둠회, 농어, 조개찜세트

3. **대한식당**
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 주소: 전라남도 광양시 광양읍 매일시장길 12-15
   - 메뉴: 국내산광양불고기, 호주산광양불고기, 소특양구이

4.

In [22]:
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

def evaluate_chain(chain, test_data):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = []

    for test_case in test_data:
        question = test_case["question"]
        expected_answer = test_case["expected_answer"]

        response = chain.invoke(question)
        generated_answer = response.strip()

        correct = expected_answer in generated_answer
        bleu_score = sentence_bleu([expected_answer.split()], generated_answer.split())
        rouge_scores = scorer.score(generated_answer, expected_answer)

        # Precision, Recall, F1 추가 계산
        precision = len(set(generated_answer.split()) & set(expected_answer.split())) / len(generated_answer.split())
        recall = len(set(generated_answer.split()) & set(expected_answer.split())) / len(expected_answer.split())
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        results.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "correct": correct,
            "bleu_score": bleu_score,
            "rouge": rouge_scores,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        })

    return results

# 평가 실행
results = evaluate_chain(chain, test_data)

# 결과 출력
for result in results:
    print(f"Question: {result['question']}")
    print(f"Expected: {result['expected_answer']}")
    print(f"Generated: {result['generated_answer']}")
    print(f"Correct: {result['correct']}")
    print(f"BLEU Score: {result['bleu_score']:.2f}")
    print(f"ROUGE: {result['rouge']}")
    print(f"Precision: {result['precision']:.2f}")
    print(f"Recall: {result['recall']:.2f}")
    print(f"F1 Score: {result['f1_score']:.2f}")
    print("-" * 30)

# 결과 저장
df_results = pd.DataFrame(results)
df_results.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")

c:\Users\qkrck\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Question: 예약 가능한 한식당 추천해줘.
Expected: 다음은 예약 가능한 한식당입니다:

1. **상주식당**
   - 음식종류: 한식(민물어패류), 추어탕
   - 전화번호: 053-425-5924
   - 주소: 대구광역시 중구 국채보상로 598-1

2. **이슬회수산**
   - 음식종류: 한식(어패류), 생선회, 조개찜
   - 전화번호: 041-633-4857
   - 주소: 충청남도 홍성군 서부면 남당항로213번길 1

3. **대한식당**
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 주소: 전라남도 광양시 광양읍 매일시장길 12-15

4. **연춘식당**
   - 음식종류: 한식(민물어패류), 장어, 한식(가금류), 닭구이
   - 전화번호: 041-545-2866
   - 주소: 충청남도 아산시 신정호길 67

5. **옥야식당**
   - 음식종류: 한식(탕/국/찌개/전골), 선지해장국
   - 전화번호: 054-853-6953
   - 주소: 경상북도 안동시 중앙시장길 7

이 외에도 다양한 한식당이 있으니, 원하는 지역이나 메뉴에 따라 선택하시면 좋겠습니다.
Generated: 다음은 예약 가능한 한식당입니다:

1. **상주식당**
   - 음식종류: 한식(민물어패류), 추어탕
   - 전화번호: 053-425-5924
   - 주소: 대구광역시 중구 국채보상로 598-1
   - 메뉴: 추어탕+밥(1만2천원)

2. **이슬회수산**
   - 음식종류: 한식(어패류), 생선회, 조개찜
   - 전화번호: 041-633-4857
   - 주소: 충청남도 홍성군 서부면 남당항로213번길 1
   - 메뉴: 해물탕, 모둠회, 농어, 조개찜세트

3. **대한식당**
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 주소: 전라남도 광양시 광양읍 매일시장길 12-15
   - 메뉴: 국내산광양불고기, 호주산광양불고기, 소특양구이

4.